In [42]:
#imports 
import os
import ollama
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [21]:
#initialization for cosmos AI

load_dotenv(override=True)

api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("key exists")
else:
    print("key does not exist")

MODEL = "gpt-4o-mini"
openai = OpenAI(
    api_key = api_key,
    base_url = "https://aiplatform.dev51.cbf.dev.paypalinc.com/cosmosai/llm/v1"
)

key exists


In [22]:
system_message = "You are a helpful assistant for Venmo application which lets user send money to friends. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [23]:
#chat function

def chat(message, history):
    messages = [{"role" : "system", "content" : system_message}] + history + [{"role" : "user", "content" : message}]
    response = openai.chat.completions.create(model = MODEL, messages = messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7876
* To create a public link, set `share=True` in `launch()`.


In [58]:
#Using tools
#the function (tool) the ai can call
money_sent = {"john" : "$50", "mia" : "$20", "judy" : "$3", "kumar" : "Rs.4/-"}

def get_money_sent(recipient):
    print(f"Tool get_money_sent called for {recipient}")
    recipient = recipient.lower()
    return money_sent.get(recipient, "Unknown")

def send_money(amount, recipient):
    return amount + " sent to "+recipient

In [86]:
# Dictionary structure that we need to pass to the AI so it can understand the tool : 

money_function = {
    "name" : "get_money_sent", 
    "description" : "Get the total amount of money sent to a particular recipient. Call this whenever you need to know the amount of money sent to a particular person, for example when a customer asks 'How much money did I send to John'",
    "parameters" : {
        "type" : "object", 
        "properties" : {
            "recipient": {
                "type": "string",
                "description": "The recipient that the customer wants to fetch the transaction history of",
            },
        },
        "required": ["desination_city"],
        "additionalProperties": False
    }
}

send_money_function = {
    "name" : "send_money",
    "description" : "Send amount to a recipient. Call this whenever the user asks you to send money to someone.",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "amount" : {
                "type" : "number", 
                "description" : "The amount of money to send to recipient",
            },
            "recipient" : {
                "type" : "string", 
                "description" : "The recipient to send money to",
            },
        },
        "required" : ["amount", "recipient"],
        "additionalProperties" : False
    }
}

In [87]:
#Add this in the list of tools available
tools = [{"type" : "function", "function" : money_function}, {"type" : "function", "function" : send_money_function}]

In [88]:
#Get OpenAI to use the tool
def chat(message, history):
    messages = [{"role" : "system", "content" : system_message}] + history + [{"role" : "user", "content" : message}]
    response = openai.chat.completions.create(model=MODEL, messages = messages, tools = tools)

    print("Response finish reason : ",response)
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        print("Response message from AI : ",message)
        response, recipient = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model = MODEL, messages = messages)

    return response.choices[0].message.content

In [89]:
#Use ollama instead
def chat_ollama(message, history):
    messages = [{"role" : "system", "content" : system_message}] + history + [{"role" : "user", "content" : message}]
    response = ollama.chat(model="llama3.2", messages = messages, tools = tools)

    print(response)
    return response['message']['content']

In [100]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    print("message tool calls : ",message.tool_calls)
    arguments = json.loads(tool_call.function.arguments)
    if(tool_call.function.name == "send_money"):
        amount = arguments.get('amount')
        recipient = arguments.get('recipient')
        response = {
            "role" : "tool",
            "content" : json.dumps({"recipient" : recipient, "amount" : amount}),
            "tool_call_id" : tool_call.id
        }
    else:  
        recipient = arguments.get('recipient')
        amount = get_money_sent(recipient)
        response = {
            "role" : "tool", 
            "content" : json.dumps({"recipient" : recipient, "amount" : amount}),
            "tool_call_id" : tool_call.id
        }
    return response, recipient

In [101]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7898
* To create a public link, set `share=True` in `launch()`.


Response finish reason :  ChatCompletion(id='chatcmpl-BdwfKsgH6j1IcT9GQkRrFDzITrOmz', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_mmd6lAIVbz987b2weSad6NOb', function=Function(arguments='{"amount":10,"recipient":"john"}', name='send_money'), type='function')]), content_filter_results={})], created=1748860002, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_57db37749c', usage=CompletionUsage(completion_tokens=19, prompt_tokens=201, total_tokens=220, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'ha

In [56]:
gr.ChatInterface(fn = chat_ollama, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7888
* To create a public link, set `share=True` in `launch()`.


model='llama3.2' created_at='2025-06-02T10:07:57.417462Z' done=True done_reason='stop' total_duration=363487292 load_duration=31199042 prompt_eval_count=251 prompt_eval_duration=160537000 eval_count=23 eval_duration=166852584 message=Message(role='assistant', content='', thinking=None, images=None, tool_calls=[ToolCall(function=Function(name='get_money_sent', arguments={'desination_city': '', 'recipient': ''}))])
